# Dự đoán nguy cơ bệnh tim mạch – Notebook hoàn chỉnh

Notebook này thực hiện toàn bộ quy trình:

1. Đọc và khám phá dữ liệu `cardio_train.csv`
2. Tiền xử lý dữ liệu (làm sạch, loại bỏ giá trị bất thường)
3. Tạo thêm một số đặc trưng mới (feature engineering)
4. Chia dữ liệu train/test
5. Xây dựng mô hình Machine Learning (Random Forest)
6. Đánh giá mô hình
7. Lưu mô hình đã train dưới dạng file `.pkl` để dùng trong backend (Python/Go)

> **Lưu ý:** Đặt file `cardio_train.csv` cùng thư mục với notebook này, hoặc sửa lại đường dẫn cho phù hợp.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import joblib

pd.set_option('display.max_columns', None)


## 1. Đọc dữ liệu

In [2]:
# Đọc file CSV với dấu phân cách là ';'
# Nếu file của bạn ở đường dẫn khác thì sửa lại cho đúng
csv_path = "cardio_train.csv"
df = pd.read_csv(csv_path, sep=';')

print("Kích thước dữ liệu:", df.shape)
df.head()


Kích thước dữ liệu: (70000, 13)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


## 2. Khám phá nhanh dữ liệu

In [3]:
# Thông tin tổng quan
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


In [4]:
# Thống kê mô tả
df.describe()


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


## 3. Tiền xử lý dữ liệu

Các bước chính:

- Loại bỏ giá trị bất hợp lý của:
  - Huyết áp tâm thu (ap_hi)
  - Huyết áp tâm trương (ap_lo)
  - Chiều cao, cân nặng
- Tạo thêm các feature mới:
  - `age_years` – tuổi (năm)
  - `bmi` – chỉ số khối cơ thể
  - `bp_ratio` – tỷ lệ huyết áp tâm thu / tâm trương


In [5]:
df_clean = df.copy()

# 3.1. Tạo tuổi theo năm
df_clean['age_years'] = df_clean['age'] / 365.25

# 3.2. Loại bỏ giá trị bất thường của huyết áp
df_clean = df_clean[(df_clean['ap_hi'] >= 50) & (df_clean['ap_hi'] <= 250)]
df_clean = df_clean[(df_clean['ap_lo'] >= 20) & (df_clean['ap_lo'] <= 200)]
df_clean = df_clean[df_clean['ap_hi'] >= df_clean['ap_lo']]

# 3.3. Loại bỏ chiều cao/cân nặng bất hợp lý
df_clean = df_clean[(df_clean['height'] >= 120) & (df_clean['height'] <= 220)]
df_clean = df_clean[(df_clean['weight'] >= 30) & (df_clean['weight'] <= 200)]

# 3.4. Tính BMI
df_clean['bmi'] = df_clean['weight'] / (df_clean['height'] / 100) ** 2

# 3.5. Loại bỏ BMI cực đoan (tuỳ chọn)
df_clean = df_clean[(df_clean['bmi'] >= 10) & (df_clean['bmi'] <= 60)]

# 3.6. Tỷ lệ huyết áp
df_clean['bp_ratio'] = df_clean['ap_hi'] / df_clean['ap_lo']

print("Kích thước dữ liệu sau khi làm sạch:", df_clean.shape)
df_clean.head()


Kích thước dữ liệu sau khi làm sạch: (68615, 16)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years,bmi,bp_ratio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0,50.357290,21.967120,1.375000
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1,55.381246,34.927679,1.555556
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1,51.627652,23.507805,1.857143
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1,48.249144,28.710479,1.500000
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0,47.841205,23.011177,1.666667


## 4. Chuẩn bị features và nhãn

In [6]:
# Nhãn (target)
y = df_clean['cardio']

# Bỏ các cột không dùng trực tiếp cho mô hình
X = df_clean.drop(columns=['cardio', 'id', 'age'])

print("Các cột feature:")
print(list(X.columns))


Các cột feature:
['gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'age_years', 'bmi', 'bp_ratio']


## 5. Chia train/test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)


Train size: (54892, 13)
Test size: (13723, 13)


## 6. Xây dựng mô hình với Pipeline (tiền xử lý + RandomForest)

In [8]:
# Các cột numeric sẽ được chuẩn hóa (StandardScaler)
numeric_features = ['age_years', 'height', 'weight', 'ap_hi', 'ap_lo', 'bmi', 'bp_ratio']

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ],
    remainder='passthrough'  # giữ nguyên các cột còn lại
)

# Mô hình RandomForest – khá mạnh và dễ dùng với dữ liệu tabular
rf_clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=8,
    min_samples_split=10,
    min_samples_leaf=5,
    n_jobs=-1,
    random_state=42
)

model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('classifier', rf_clf)
])

model


,steps,"[('preprocess', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


## 7. Train mô hình

In [10]:
model.fit(X_train, y_train)
print("Đã train xong mô hình RandomForest.")


Đã train xong mô hình RandomForest.


## 8. Đánh giá mô hình

In [11]:
# Dự đoán trên tập test
y_pred = model.predict(X_test)

# Nếu classifier hỗ trợ predict_proba thì tính thêm ROC AUC
if hasattr(model.named_steps['classifier'], "predict_proba"):
    y_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    print("ROC AUC:", round(auc, 4))

print("\nClassification report:")
print(classification_report(y_test, y_pred))


ROC AUC: 0.7959

Classification report:
              precision    recall  f1-score   support

           0       0.70      0.80      0.75      6934
           1       0.76      0.65      0.70      6789

    accuracy                           0.73     13723
   macro avg       0.73      0.73      0.73     13723
weighted avg       0.73      0.73      0.73     13723



## 9. Lưu mô hình thành file `.pkl`

In [12]:
# Lưu cả pipeline (tiền xử lý + model) vào một file duy nhất
model_filename = "heart_disease_model.pkl"
joblib.dump(model, model_filename)

print(f"Đã lưu mô hình vào file: {model_filename}")


Đã lưu mô hình vào file: heart_disease_model.pkl


## 10. Ví dụ load lại model và dự đoán cho 1 mẫu

In [13]:
# Ví dụ: load lại mô hình và predict cho 1 dòng bất kỳ trong X_test

loaded_model = joblib.load("heart_disease_model.pkl")

sample = X_test.iloc[[0]]
print("Sample features:")
print(sample)

sample_pred_proba = loaded_model.predict_proba(sample)[0, 1]
sample_pred_label = loaded_model.predict(sample)[0]

print("\nXác suất mắc bệnh tim (cardio=1):", round(sample_pred_proba, 4))
print("Dự đoán nhãn:", int(sample_pred_label))


Sample features:
       gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  alco  \
22306       1     169    56.0    120     80            1     1      0     0   

       active  age_years        bmi  bp_ratio  
22306       0  50.425736  19.607157       1.5  

Xác suất mắc bệnh tim (cardio=1): 0.2507
Dự đoán nhãn: 0
